ライブラリのインポート

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import sys, os

# Mstudy ディレクトリをパスに追加
sys.path.append(os.path.abspath(".."))

from PythonWorks import AMD_Tools4 as amd

FieldData.dbから緯度経度などのデータ取得のための関数

In [2]:
def get_locations_from_db():
    """
    FieldData.dbに接続し、Questionaireテーブルからデータをフィルタリングして、
    処理対象となる場所、年、緯度、経度のDataFrameを返す関数。
    """
    base_dir = os.getcwd()  # 現在の実行ディレクトリ
    db_path = os.path.join(base_dir, 'FieldData.db')
    locations_df = None
    try:
        conn = sqlite3.connect(db_path)
        query = "SELECT * FROM Questionaire"
        df_original = pd.read_sql_query(query, conn)
    except sqlite3.Error as e:
        print(f"データベースエラー: {e}")
        return None
    finally:
        if 'conn' in locals() and conn:
            conn.close()

    df_original = df_original.replace(r'^\s*$', np.nan, regex=True)
    columns_to_check = ['lat', 'lon', 'yield', 'seed_date', 'harvest_date']

    if all(col in df_original.columns for col in columns_to_check):
        df_filtered = df_original.dropna(subset=columns_to_check)
        locations_df = df_filtered[['place', 'year', 'lat', 'lon']].copy()
    else:
        print("エラー: テーブルに必須の列が見つかりません。")

    return locations_df

メッシュからの気象情報取得のための関数

In [3]:
def fetch_weather_for_location(doko, itsu):
    """
    指定された1地点（doko）と期間（itsu）の複数の気象要素を取得し、
    DataFrameとして返す関数。
    """
    # 取得したい気象要素のリスト
    elements = {
        'TMP_mea': '日平均気温',
        'TMP_max': '日最高気温',
        'TMP_min': '日最低気温',
        'APCPRA': '日積算降水量',
        'OPR': '1mm以上の降水の有無',
        'SSD': '日照時間',
        'GSR': '全天日射量',
        'DLR': '下向き長波放射量',
        'RH': '日平均相対湿度',
        'WIND': '日平均風速',
        'SD': '積雪深',
        'SWE': '積雪相当水量',
        'SFW': '日降雪相当水量'
    }
    weather_data_dict = {}
    tim = None

    for code, user_name in elements.items():
        data, temp_tim, _, _, name, unit = amd.GetMetData(code, itsu, doko, namuni=True)
        series_data = data[:, 0, 0]
        new_column_name = f"{user_name} ({unit})"
        weather_data_dict[new_column_name] = series_data
        
        if tim is None:
            tim = temp_tim
            
    return pd.DataFrame(weather_data_dict, index=tim)

In [4]:
# 1. データベースから全地点のリストを取得
locations_to_process = get_locations_from_db()

# 2. 年ごとにDataFrameを分割して辞書に格納
locations_by_year = {}
if locations_to_process is not None:
    for year in [2015, 2016, 2017, 2018]:
        locations_by_year[year] = locations_to_process[locations_to_process['year'] == year].copy()
        print(f"{year}年の対象地点数: {len(locations_by_year[year])}")

# 最終的に結合するためのリストを準備
all_weather_dfs = []

2015年の対象地点数: 136
2016年の対象地点数: 156
2017年の対象地点数: 133
2018年の対象地点数: 85


In [5]:
# --- 2015年の処理 ---
year = 2015
itsu = ['2015-04-01', '2016-03-24']
yearly_dfs = []

print(f"\n--- {year}年の気象データ取得を開始します ---")
for index, row in locations_by_year[year].iterrows():
    print(f"処理中: {row['place']} ({year})")
    weather_df = fetch_weather_for_location([row['lat'], row['lat'], row['lon'], row['lon']], itsu)
    weather_df['place'] = row['place']
    weather_df['year'] = year
    yearly_dfs.append(weather_df)

# この年のデータを結合して、全体のリストに追加
if yearly_dfs:
    all_weather_dfs.append(pd.concat(yearly_dfs))
    print(f"--- {year}年の処理が完了しました ---")


--- 2015年の気象データ取得を開始します ---
処理中: B1 (2015)
TMP_mea (359, 1, 1) Tile
TMP_max (359, 1, 1) Tile
TMP_min (359, 1, 1) Tile
APCPRA (359, 1, 1) Tile
OPR (359, 1, 1) Tile
SSD (359, 1, 1) Tile
GSR (359, 1, 1) Tile
DLR (359, 1, 1) Tile
RH (359, 1, 1) Tile
WIND (359, 1, 1) Tile
SD (359, 1, 1) Tile
SWE (359, 1, 1) Tile
SFW (359, 1, 1) Tile
処理中: B2 (2015)
TMP_mea (359, 1, 1) Tile
TMP_max (359, 1, 1) Tile
TMP_min (359, 1, 1) Tile
APCPRA (359, 1, 1) Tile
OPR (359, 1, 1) Tile
SSD (359, 1, 1) Tile
GSR (359, 1, 1) Tile
DLR (359, 1, 1) Tile
RH (359, 1, 1) Tile
WIND (359, 1, 1) Tile
SD (359, 1, 1) Tile
SWE (359, 1, 1) Tile
SFW (359, 1, 1) Tile
処理中: B3 (2015)
TMP_mea (359, 1, 1) Tile
TMP_max (359, 1, 1) Tile
TMP_min (359, 1, 1) Tile
APCPRA (359, 1, 1) Tile
OPR (359, 1, 1) Tile
SSD (359, 1, 1) Tile
GSR (359, 1, 1) Tile
DLR (359, 1, 1) Tile
RH (359, 1, 1) Tile
WIND (359, 1, 1) Tile
SD (359, 1, 1) Tile
SWE (359, 1, 1) Tile
SFW (359, 1, 1) Tile
処理中: B4 (2015)
TMP_mea (359, 1, 1) Tile
TMP_max (359, 1, 1) Tile


In [6]:
# --- 2016年の処理 ---
year = 2016
itsu = ['2016-04-01', '2017-03-24']
yearly_dfs = []

print(f"\n--- {year}年の気象データ取得を開始します ---")
for index, row in locations_by_year[year].iterrows():
    print(f"処理中: {row['place']} ({year})")
    weather_df = fetch_weather_for_location([row['lat'], row['lat'], row['lon'], row['lon']], itsu)
    weather_df['place'] = row['place']
    weather_df['year'] = year
    yearly_dfs.append(weather_df)

if yearly_dfs:
    all_weather_dfs.append(pd.concat(yearly_dfs))
    print(f"--- {year}年の処理が完了しました ---")


--- 2016年の気象データ取得を開始します ---
処理中: A1 (2016)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: A4 (2016)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: A5 (2016)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: A6 (2016)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile


In [7]:
# --- 2017年の処理 ---
year = 2017
itsu = ['2017-04-01', '2018-03-24']
yearly_dfs = []

print(f"\n--- {year}年の気象データ取得を開始します ---")
for index, row in locations_by_year[year].iterrows():
    print(f"処理中: {row['place']} ({year})")
    weather_df = fetch_weather_for_location([row['lat'], row['lat'], row['lon'], row['lon']], itsu)
    weather_df['place'] = row['place']
    weather_df['year'] = year
    yearly_dfs.append(weather_df)

if yearly_dfs:
    all_weather_dfs.append(pd.concat(yearly_dfs))
    print(f"--- {year}年の処理が完了しました ---")


--- 2017年の気象データ取得を開始します ---
処理中: B1 (2017)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: B2 (2017)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: B3 (2017)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: B4 (2017)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile


In [8]:
# --- 2018年の処理 ---
year = 2018
itsu = ['2018-04-01', '2019-03-24']
yearly_dfs = []

print(f"\n--- {year}年の気象データ取得を開始します ---")
for index, row in locations_by_year[year].iterrows():
    print(f"処理中: {row['place']} ({year})")
    weather_df = fetch_weather_for_location([row['lat'], row['lat'], row['lon'], row['lon']], itsu)
    weather_df['place'] = row['place']
    weather_df['year'] = year
    yearly_dfs.append(weather_df)

if yearly_dfs:
    all_weather_dfs.append(pd.concat(yearly_dfs))
    print(f"--- {year}年の処理が完了しました ---")


--- 2018年の気象データ取得を開始します ---
処理中: B01 (2018)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: B02 (2018)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: C01 (2018)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) Tile
TMP_min (358, 1, 1) Tile
APCPRA (358, 1, 1) Tile
OPR (358, 1, 1) Tile
SSD (358, 1, 1) Tile
GSR (358, 1, 1) Tile
DLR (358, 1, 1) Tile
RH (358, 1, 1) Tile
WIND (358, 1, 1) Tile
SD (358, 1, 1) Tile
SWE (358, 1, 1) Tile
SFW (358, 1, 1) Tile
処理中: C04 (2018)
TMP_mea (358, 1, 1) Tile
TMP_max (358, 1, 1) T

In [1]:
if all_weather_dfs:
    # 全ての年のDataFrameを一つに結合
    final_df = pd.concat(all_weather_dfs)
    
    # indexになっている日付を列に変換
    final_df = final_df.reset_index().rename(columns={'index': '日付'})
    
    # 列の順番を並べ替え
    weather_cols = [col for col in final_df.columns if col not in ['year', 'place', '日付']]
    new_order = ['year', 'place', '日付'] + weather_cols
    final_df = final_df[new_order]

    # 最終結果の表示と保存
    print("\n" + "="*50 + "\n")
    print("--- 全地点の気象データを結合した最終結果 ---")
    print(final_df)
    print(final_df.columns)
    
    output_filename = 'all_locations_weather.csv'
    final_df.to_csv(output_filename, encoding='utf-8-sig', index=False)
    print(f"\n全データを'{output_filename}'に保存しました。")
else:
    print("\n気象データを取得できる地点がありませんでした。")

NameError: name 'all_weather_dfs' is not defined

In [10]:
# 'final_df'が存在し、空でないことを確認
if 'final_df' in locals() and not final_df.empty:
    
    # --- 1. 列名を日本語から英語コードに変換 ---
    # 元のDataFrameを変更しないようにコピーを作成
    df_for_db = final_df.copy()

    # 日本語名から英語コードへの対応表
    rename_map = {
        '日付': 'date', # '日付'列も英語に
    }
    code_to_jp_name = {
        'TMP_mea': '日平均気温',
        'TMP_max': '日最高気温',
        'TMP_min': '日最低気温',
        'APCPRA': '日積算降水量',
        'OPR': '1mm以上の降水の有無',
        'SSD': '日照時間',
        'GSR': '全天日射量',
        'DLR': '下向き長波放射量',
        'RH': '日平均相対湿度',
        'WIND': '日平均風速',
        'SD': '積雪深',
        'SWE': '積雪相当水量',
        'SFW': '日降雪相当水量'
    }

    # 現在の列名（例: '日最高気温 (degC)'）と日本語名を照合して変換マップを完成させる
    for code, jp_name in code_to_jp_name.items():
        for col in df_for_db.columns:
            if col.startswith(jp_name):
                rename_map[col] = code
                break

    # DataFrameの列名を一括で変更
    df_for_db.rename(columns=rename_map, inplace=True)
    
    print("\n" + "="*50 + "\n")
    print("--- データベース保存用に列名を英語に変換しました ---")
    print("新しい列名:", df_for_db.columns.tolist())
    
    # --- 2. データベースファイルに保存 ---
    db_filename = 'weather_database.db'
    table_name = 'weather_data'
    
    try:
        # データベースファイルに接続（ファイルがなければ新規作成）
        conn = sqlite3.connect(db_filename)
        
        # DataFrameの内容をデータベースのテーブルに書き込み
        df_for_db.to_sql(
            name=table_name,
            con=conn,
            if_exists='replace', # もし同名のテーブルがあれば、上書きする
            index=False        # DataFrameのインデックスはDBに保存しない
        )
        
        print(f"\nDataFrameが'{db_filename}'ファイル内の'{table_name}'テーブルに正常に保存されました。")
        
    except Exception as e:
        print(f"\nデータベースへの保存中にエラーが発生しました: {e}")
        
    finally:
        # 接続を閉じる
        if 'conn' in locals() and conn:
            conn.close()

else:
    print("\nfinal_dfが作成されていないか、空のため、データベースファイルは作成されませんでした。")



--- データベース保存用に列名を英語に変換しました ---
新しい列名: ['year', 'place', 'date', 'TMP_mea', 'TMP_max', 'TMP_min', 'APCPRA', 'OPR', 'SSD', 'GSR', 'DLR', 'RH', 'WIND', 'SD', 'SWE', 'SFW', '1mm以上の降水の有無 ()']

DataFrameが'weather_database.db'ファイル内の'weather_data'テーブルに正常に保存されました。


# 注意
2017年以降のOPR列に格納されるはずの値が1mm以上の降水量の有無()という列に格納されている。これはchange_DB.pyで対応しているので、このコードを実行後当該プログラムを実行してください。また、その時SQLの処理の都合上1mm以上の降水量の有無()の列名をOPRoriという名前に変えてから実行してください。